In [21]:

case class CoffeeMachine(resources: CoffeeMachine.Resources, money: CoffeeMachine.Coins)

object CoffeeMachine: 

    case class Resources(water: Int, milk: Int, coffee: Int)

    case class Coins(quarters: Int, dimes: Int, nickles: Int, pennies: Int)

    object Coins:

        def total(coins: Coins): BigDecimal = 
            coins.quarters * 0.25 + 
            coins.dimes * 0.10 + 
            coins.nickles * 0.05 + 
            coins.pennies * 0.01

        def toCoins(total: BigDecimal): Coins = 
            val quarters: Int = (total / 0.25).toInt
            val total1: BigDecimal = total - quarters * 0.25
            val dimes: Int = (total1 / 0.10).toInt 
            val total2: BigDecimal = total1 - dimes * 0.10
            val nickles: Int = (total2 / 0.05).toInt
            val total3: BigDecimal = total2 - nickles * 0.05 
            val pennies: Int = (total3 / 0.01).toInt 
            Coins(quarters, dimes, nickles, pennies)

        extension (c: Coins)
            def +(c2: Coins): Coins = 
                (c, c2) match 
                    case (Coins(q1, d1, n1, p1), Coins(q2, d2, n2, p2)) => 
                        Coins(q1+q2, d1+d2, n1+n2, p1+p2)
            def -(c2: Coins): Coins = 
                (c, c2) match 
                    case (Coins(q1, d1, n1, p1), Coins(q2, d2, n2, p2)) => 
                        Coins(q1-q2, d1-d2, n1-n2, p1-p2)

    enum Drink(val resources: Resources, val price: Double): 
        case Latte extends Drink(Resources(1,3,1), 0.5)
        case Expresso extends Drink(Resources(1,0,1), 0.7)
        case Cappuccino extends Drink(Resources(1,3,3), 0.9)

    enum Command[A]: 
        case Order(drink: Drink, coins: Coins) extends Command[OrderResult]
        case Report extends Command[(Resources, BigDecimal)]
        case Off extends Command[Unit]

    import Command._ 

    enum OrderResult:
        case NotEnoughResources
        case NotEnoughCoins
        case Change(c: Coins)

    def execute[A](command: Command[A])(machine: CoffeeMachine): (CoffeeMachine, A) = 
        command match 
            case Order(d, c) => order(d, c)(machine)
            case Report => (machine, report(machine))
            case Off => (machine, ()) // free resources

    def report(machine: CoffeeMachine): (Resources, BigDecimal) = 
        (machine.resources, Coins.total(machine.money))

    def order(drink: Drink, coins: Coins)(
        machine: CoffeeMachine): (CoffeeMachine, OrderResult) = 

        def prepareCoffee(drink: Drink)(machine: CoffeeMachine): Option[CoffeeMachine] = 

            def checkResources(drink: Drink): Boolean = 
                machine.resources.water >= drink.resources.water && 
                machine.resources.milk >= drink.resources.milk && 
                machine.resources.coffee >= drink.resources.coffee 

            if !checkResources(drink) then None 
            else machine match 
                case CoffeeMachine(Resources(water, milk, coffee), coins) => 
                    Some(CoffeeMachine(
                        Resources(
                            water - drink.resources.water, 
                            milk - drink.resources.milk,
                            coffee - drink.resources.coffee), 
                        coins))

        def processCoins(drink: Drink, coins: Coins)(
            machine: CoffeeMachine): Option[(CoffeeMachine, OrderResult.Change)] = 

            def checkCoins(drink: Drink, coins: Coins): Boolean = 
                Coins.total(coins) >= drink.price

            if !checkCoins(drink, coins) then None
            else machine match 
                case CoffeeMachine(r, coinsM) => 
                    val totalChange: BigDecimal = Coins.total(coins) - drink.price
                    val changeInCoins: Coins = Coins.toCoins(totalChange)
                    Some((CoffeeMachine(r, coinsM + coins - changeInCoins), 
                          OrderResult.Change(changeInCoins)))


        prepareCoffee(drink)(machine) match 
            case None => (machine, OrderResult.NotEnoughResources)
            case Some(machine2) => 
                processCoins(drink, coins)(machine2) match 
                    case None => (machine, OrderResult.NotEnoughCoins)
                    case Some(result) => result

    

    

defined class CoffeeMachine
defined object CoffeeMachine

In [15]:
import CoffeeMachine._

import CoffeeMachine._


In [16]:
val m1 = CoffeeMachine(Resources(25,25,25), Coins(50, 50, 50, 50))

m1: CoffeeMachine = CoffeeMachine(
  resources = Resources(water = 25, milk = 25, coffee = 25),
  money = Coins(quarters = 50, dimes = 50, nickles = 50, pennies = 50)
)

In [17]:
val (m2, r2) = execute(Command.Order(Drink.Latte, Coins(1,2,1,0)))(m1)

m2: CoffeeMachine = CoffeeMachine(
  resources = Resources(water = 24, milk = 22, coffee = 24),
  money = Coins(quarters = 51, dimes = 52, nickles = 51, pennies = 50)
)
r2: OrderResult = Change(
  c = Coins(quarters = 0, dimes = 0, nickles = 0, pennies = 0)
)

In [18]:
val (m3, r3) = execute(Command.Order(Drink.Cappuccino, Coins(4,0,0,0)))(m2)

m3: CoffeeMachine = CoffeeMachine(
  resources = Resources(water = 23, milk = 19, coffee = 21),
  money = Coins(quarters = 55, dimes = 51, nickles = 51, pennies = 50)
)
r3: OrderResult = Change(
  c = Coins(quarters = 0, dimes = 1, nickles = 0, pennies = 0)
)

In [19]:
val (m4, report) = execute(Command.Report)(m3)

m4: CoffeeMachine = CoffeeMachine(
  resources = Resources(water = 23, milk = 19, coffee = 21),
  money = Coins(quarters = 55, dimes = 51, nickles = 51, pennies = 50)
)
report: (Resources, BigDecimal) = (
  Resources(water = 23, milk = 19, coffee = 21),
  21.900000000000002
)